# Homework 2 Evolutionary Algorithms.
This file will include all the necessary code for evolutionary algorithm homework. The task is described as "In this homework, you will perform experiments on evolutionary algorithm and draw conclusions from the experimental results. The task is to create an image made of filled circles, visually similar to a given RGB source image (painting.png)"

## Pseudo code
Initialize population with <num_inds> individuals each having <num_genes> genes
While not all generations (<num_generations>) are computed:
Evaluate all the individuals
Select individuals
Do crossover on some individuals
Mutate some individuals

In [1]:
# "Individual" class definition for evolutionary algoritm. There will be on chromosome and N number of genes. Each gene will have center coordinates (x,y), Radius, and RGB color.

import random
import math
import numpy as np
import cv2

class Individual:
    def __init__(self, num_genes, image_size):
        self.num_genes = num_genes
        self.image_size = image_size
        self.chromosome = []
        self.fitness = 0
        self.elite = False

        for i in range(num_genes):
            x = random.randint(0, image_size[0])
            y = random.randint(0, image_size[1])
            r = random.randint(0, 50)
            color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
            alpha = random.random()

            self.chromosome.append([x, y, r, color, alpha])

    def mutate(self, mutation_probability, guidance = None):
        if not self.elite:
            if guidance is None:
                for i in range(self.num_genes):
                    if random.random() < mutation_probability:
                        self.chromosome[i][0] = random.randint(0, self.image_size[0])
                        self.chromosome[i][1] = random.randint(0, self.image_size[1])
                        self.chromosome[i][2] = random.randint(0, 50)
                        self.chromosome[i][3] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
            else:
                #Guided mutation, deviate x,y, radius, color and alpha around their previous values
                for i in range(self.num_genes):
                    if random.random() < mutation_probability:
                        self.chromosome[i][0] = int(self.chromosome[i][0] + (self.image_size[0]/2)*random.uniform(-1,1))
                        self.chromosome[i][1] = int(self.chromosome[i][1] + (self.image_size[1]/2)*random.uniform(-1,1))
                        self.chromosome[i][2] = int(self.chromosome[i][2] + 10*random.uniform(-1,1))
                        self.chromosome[i][3][0] = self.chromosome[i][3][0] + 64*random.uniform(-1, 1)
                        if self.chromosome[i][3][0] < 0:
                            self.chromosome[i][3][0] = 0
                        elif self.chromosome[i][3][0] > 255:
                            self.chromosome[i][3][0] = 255

                        self.chromosome[i][3][1] = self.chromosome[i][3][1] + 64*random.uniform(-1, 1)
                        if self.chromosome[i][3][1] < 0:
                            self.chromosome[i][3][1] = 0
                        elif self.chromosome[i][3][1] > 255:
                            self.chromosome[i][3][1] = 255
                        
                        self.chromosome[i][3][2] = self.chromosome[i][3][2] + 64*random.uniform(-1, 1)
                        if self.chromosome[i][3][2] < 0:
                            self.chromosome[i][3][2] = 0
                        elif self.chromosome[i][3][2] > 255:
                            self.chromosome[i][3][2] = 255

                        self.chromosome[i][3] = [int(x) for x in self.chromosome[i][3]]
                        
                        self.chromosome[i][4] = self.chromosome[i][4] + 0.25*random.uniform(-1, 1)
                        if self.chromosome[i][4] < 0:
                            self.chromosome[i][4] = 0
                        elif self.chromosome[i][4] > 1:
                            self.chromosome[i][4] = 1

                        # #Check if the gene is still in the image
                        # if self.chromosome[i][0] - self.chromosome[i][2] < 0 or self.chromosome[i][0] + self.chromosome[i][2] > self.image_size[0] or self.chromosome[i][1] - self.chromosome[i][2] < 0 or self.chromosome[i][1] + self.chromosome[i][2] > self.image_size[1]:
                        #     # randomly initialize the gene and check again
                        #     self.chromosome[i][0] = random.randint(0, self.image_size[0])
                        #     self.chromosome[i][1] = random.randint(0, self.image_size[1])
                        #     self.chromosome[i][2] = random.randint(0, 50)
    def draw(self):
        # First sort the genes by radius
        self.chromosome.sort(key=lambda x: x[2])

        # Create a blank image white background
        image = np.ones((self.image_size[1], self.image_size[0], 3), np.uint8)*255
        
        for gene in self.chromosome:
            #check if the circle is visible in the image, center does not have to be in the image but the circle should be visible
            if gene[0] - gene[2] < 0 or gene[0] + gene[2] > self.image_size[0] or gene[1] - gene[2] < 0 or gene[1] + gene[2] > self.image_size[1]:
                # randomly initialize the gene and check again
               
                gene[0] = random.randint(0, self.image_size[0])
                gene[1] = random.randint(0, self.image_size[1])
                gene[2] = random.randint(0, 50)
            overlay = image.copy()
            cv2.circle(overlay, (gene[0], gene[1]), gene[2], gene[3], -1)
            image = cv2.addWeighted(overlay, gene[4], image, 1 - gene[4], 0)
        return image

    def calculate_fitness(self, target):
        image = self.draw()
        # Calculate the difference between the target image and the generated image
        diff = cv2.absdiff(target, image)
        # take the square of the difference
        diff = diff**2
        # sum of the squared differences
        self.fitness = -np.sum(diff)

    def crossover(self, partner):
        child1 = Individual(self.num_genes, self.image_size)
        child2 = Individual(self.num_genes, self.image_size)

        for i in range(self.num_genes):
            if random.random() < 0.5:
                child1.chromosome[i] = self.chromosome[i]
                child2.chromosome[i] = partner.chromosome[i]
            else:
                child1.chromosome[i] = partner.chromosome[i]
                child2.chromosome[i] = self.chromosome[i]
        return child1, child2

In [2]:
# Popoulation class definition

class Population:
    def __init__(self, num_individuals, num_genes, image_size, num_elites, num_parents, tm_size):
        self.num_individuals = num_individuals
        self.num_genes = num_genes
        self.image_size = image_size
        self.num_elites = num_elites
        self.num_parents = num_parents
        self.tm_size = tm_size

        self.individuals = []
        self.target = np.zeros((self.image_size[1], self.image_size[0], 3), np.uint8)
        self.target[:] = 255

        for i in range(self.num_individuals):
            self.individuals.append(Individual(self.num_genes, self.image_size))

    def selection(self):
        self.individuals.sort(key=lambda x: x.fitness)
        new_individuals = []
        # Mark the best individuals as elite
        for i in range(self.num_elites):
            self.individuals[i].elite = True
            new_individuals.append(self.individuals[i])
        # Select the rest of the individuals using tournament selection.
        num_groups = math.floor((len(self.individuals) - self.num_elites) / self.tm_size)
        selectable_individuals = self.individuals[self.num_elites:]

        # draw num_groups number of tournaments do not draw one elemnt multiple times
        for i in range(int(num_groups)):
            group = random.sample(selectable_individuals, self.tm_size)
            # pop group from selectable_individuals
            selectable_individuals = list(set(selectable_individuals) - set(group))

            group.sort(key=lambda x: x.fitness)
            new_individuals.append(group[0])
        
        parentable_individuals = list(set(self.individuals) - set(new_individuals))
        parentable_individuals.sort(key=lambda x: x.fitness)
        self.parents = parentable_individuals[:self.num_parents]
        
        self.individuals = new_individuals
        
    def crossover(self):
        # parents will create new individuals by crossover. Two parents will create two children
        new_individuals = []
        for i in range(self.num_parents):
            parent1 = random.choice(self.parents)
            parent2 = random.choice(self.parents)
            child1, child2 = parent1.crossover(parent2)

            new_individuals.append(child1)
            new_individuals.append(child2)
        self.individuals.extend(new_individuals)

    def mutation(self, mutation_probability):
        #check if the individual is an elite, if so do not mutate
        for individual in self.individuals: 
            individual.mutate(mutation_probability)

    def evaluate(self):
        for individual in self.individuals:
            individual.calculate_fitness(self.target)

    def get_best(self):
        self.individuals.sort(key=lambda x: x.fitness)
        return self.individuals[0]

    def get_average_fitness(self):
        return sum([x.fitness for x in self.individuals]) / self.num_individuals

In [3]:
num_individuals = 5
num_genes = 15
tournament_size = 2
frac_elites = 0.04
num_elites = int(num_individuals*frac_elites)
frac_parents = 0.15
num_parents = int(num_individuals*frac_parents)
mutation_probability = 0.1
mutataion_type = "guided"

# load input image

target = cv2.imread("painting.png")
image_size = (target.shape[1], target.shape[0])

pop = Population(num_individuals, num_genes, image_size, num_elites, num_parents, tournament_size)


In [4]:
pop.evaluate()
image = pop.individuals[0].draw()

from matplotlib import pyplot as plt
#show image in line
def show_rgb_image(image, title=None, conversion=cv2.COLOR_BGR2RGB):

    # Converts from one colour space to the other. this is needed as RGB
    # is not the default colour space for OpenCV
    image = cv2.cvtColor(image, conversion)

    # Show the image
    plt.imshow(image)

    # remove the axis / ticks for a clean looking image
    plt.xticks([])
    plt.yticks([])

    # if a title is provided, show it
    if title is not None:
        plt.title(title)

    plt.show()


/tmp/ipykernel_26924/3801918765.py:100: RuntimeWarning: overflow encountered in scalar negative
  self.fitness = -np.sum(diff)


In [5]:
# import time
# for i in range(10):
#     show_rgb_image(pop.individuals[i].draw(), "Initial Image")
#     time.sleep(1)

In [6]:
pop = Population(num_individuals, num_genes, image_size, num_elites, num_parents, tournament_size)

from tqdm import tqdm
# iterate over generations
for i in tqdm(range(10)):
    pop.selection()
    pop.crossover()
    pop.mutation(mutation_probability)
    pop.evaluate()
    print("Generation: ", i, "Average fitness: ", pop.get_average_fitness(), "Best fitness: ", pop.get_best().fitness)
    #reset elite status
    for individual in pop.individuals:
        individual.elite = False
    print("num individuals left",len(pop.individuals))

  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_26924/3801918765.py:100: RuntimeWarning: overflow encountered in scalar negative
  self.fitness = -np.sum(diff)
 20%|██        | 2/10 [00:00<00:00, 129.37it/s]


Generation:  0 Average fitness:  7.378697629481757e+18 Best fitness:  18446744073703634521
2
Generation:  1 Average fitness:  3.689348814740844e+18 Best fitness:  18446744073704220956
1


IndexError: list index out of range